In [1]:
import numpy as np
import joblib

%matplotlib widget
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import sys
sys.path.append("../")
sys.path.append("../../RARinterpret")
import lgpc
import RARinterpret

%load_ext autoreload
%autoreload 2


X, y, weights, features, indxs = (joblib.load('../../RARinterpret/results/processed_cat.p')[p]
                           for p in ("X", "y", "weights", "features", "indxs"))

In [4]:
n_splits = 500
# GroupShuffle split mask
test_masks = RARinterpret.train_test_shuffle_mask(indxs, n_splits=n_splits, test_size=42,
                                                  random_state=42)

# Initialise arrays to store the data
N = y.size
_x = np.arange(N)
Xeval = np.linspace(X.min(), X.max(), 500).reshape(-1,1)
residuals = np.full((n_splits, N), np.nan)
scores = np.full(n_splits, np.nan)
yevals = np.full((n_splits, Xeval.shape[0]), np.nan)

gpr = GaussianProcessRegressor(kernel=RBF() + WhiteKernel(), n_restarts_optimizer=1, normalize_y=True)


# for i in tqdm(range(n_splits)):
#     _res, _score, _yeval = RARinterpret.residuals_gpr(
#         gpr, X, y, _x[~test_masks[i, :]], _x[test_masks[i, :]], weights, Xeval)
#     residuals[i, :] = _res
#     scores[i] = _score
#     yevals[i, :] = _yeval
    

# xeval = Xeval.reshape(-1)

In [5]:
i = 0
res, score, yeval = lgpc.residuals_gpr(gpr, X, y, _x[~test_masks[i, :]], _x[test_masks[i, :]], weights, Xeval)

AttributeError: module 'lgpc' has no attribute 'residuals_gpr'

In [6]:
lgpc

<module 'lgpc' (namespace)>